# 数据的合并
![](./picture/5-1.png)
数据合并主要包括下面两种操作：  
轴向连接（concatenation）：pd.concat()可以沿一个轴将多个DataFrame对象连接在一起，形成一个新的DataFrame对象。  
融合（merging）：pd.merge()方法可以根据一个或多个键将不同DataFrame中的行连接起来。  
concat()函数可以将数据根据不同的轴作进行合并。我们先看一下concat()的常用参数：  
代码片段  
```
pd.concat(objs, axis=0, join='outer')
```
objs: series、dataframe或者是panel构成的序列list。  
axis： 需要合并链接的轴，0是行，1是列，默认是0。    
join：连接的方式 inner，或者outer，默认是outer。  

In [15]:
import pandas as pd
dict1={
    'A': ['A0', 'A1', 'A2', 'A3'],
    'B': ['B0', 'B1', 'B2', 'B3'],
    'C': ['C0', 'C1', 'C2', 'C3']}
df1=pd.DataFrame(dict1)
# print(df1)
# df1.head(3)

dict2={
    'B': ['B0', 'B1', 'B2', 'B3'],
    'C': ['C0', 'C1', 'C2', 'C3'],
    'D': ['D0', 'D1', 'D2', 'D3']}
df2=pd.DataFrame(dict2)
# print(df2)
# df2.head(3)

# result1 = pd.concat([df1,df2], axis = 0, join="outer")
# result1.head()

# result2 = pd.concat([df1,df2], axis = 0, join="inner")
# result2.head()


# result3 = pd.concat([df1,df2], axis = 1, join="outer")
# result3.head()

result4 = pd.concat([df1,df2], axis = 1, join="inner")
result4.head()


,A,B,C,B,C,D
0,A0,B0,C0,B0,C0,D0
1,A1,B1,C1,B1,C1,D1
2,A2,B2,C2,B2,C2,D2
3,A3,B3,C3,B3,C3,D3


当concat()使用默认参数合并df1和df2时：  
代码片段  
```
pd.concat(objs, axis=0, join='outer')
```
![](./picture/5-2.png)
通过上面的结果可以发现，当join='outer'，axis参数为0时，列进行并集处理，纵向表拼接，缺失值由NaN填充，并且会保留原有数据的行索引。  
如果两个表的index都没有实际含义，使用ignore_index参数，置true，重新整理一个新的index。  
代码片段  
```
pd.concat([df1,df2],axis=0,join='outer',ignore_index=True)
```
当concat()的axis参数为1合并df1和df2时：  
代码片段  
```
pd.concat([df1,df2],axis=1,join='outer')
```
![](./picture/5-3.png)
当join='outer'，axis参数为1时，行进行并集处理，横向表拼接，缺失值由NaN填充。  
当concat()的join参数为inner时合并df1和df2时：  
代码片段  
```
pd.concat([df1,df2],axis=0,join='inner')
```
![](./picture/5-4.png)
通过上面的结果可以看出，如果为inner，得到的是两表的交集，如果是outer，得到的是两表的并集。  
下面大家复制上面的代码到下面的代码框运行，验证的一下自己的理解：  

merge()函数通过指定连接键拼接列数据，我们先看一下merge的常用参数：  
代码片段  
```
merge(left, right, how='inner', on=None)
```
参数介绍  
left和right：两个要合并的DataFrame  
how：连接方式，有inner、left、right、outer，默认为inner  
on：指的是用于连接的列索引名称，必须存在于左右两个DataFrame中，如果没有指定且其他参数也没有指定，则以两个DataFrame列名交集作为连接键  
运行下面代码，了解数据的基本情况。  

In [18]:
import pandas as pd
left = pd.DataFrame({'key':['a','b','b','d'],'data1':range(4)})
# print(left)

right = pd.DataFrame({'key':['a','b','c'],'data2':range(3)})
# print(right)

# pd.merge(left, right, how='inner', on=None)
result = pd.merge(left,right,on=['key'],how='outer')
result.head(10)

,key,data1,data2
0,a,0.0,0.0
1,b,1.0,1.0
2,b,2.0,1.0
3,d,3.0,NaN
4,c,NaN,2.0


当merge()使用默认参数连接两个DataFrame时：  
代码片段  
```
pd.merge(left, right)
```
![](./picture/5-5.png)
merge()默认做inner连接，并且使用两个DataFrame的列名交集（key）作为连接键，同样，最终连接的数据也是两个DataFramekey列数据的交集。  
当两个DataFram使用做outer连接时：  
代码片段  
```
pd.merge(left,right,on=['key'],how='outer')
```
![](./picture/5-6.png)
当merge()做outer连接时最终连接的数据是两个DataFramekey列数据的并集，缺失的内容由NaN填充。  
当两个DataFram使用left做连接时：  
代码片段  
```
pd.merge(left,right,on=['key'],how='left')
```
![](./picture/5-7.png)
当merge()做left连接时，最终连接的数据将以left数据的链接建为准合并两个数据的列数据，缺失的内容由NaN填充  
那么，当merge()做right连接时，最终的链接数据是什么样呢？运行下面的代码，验证你的想法：  

In [21]:
import pandas as pd
left = pd.DataFrame({'key':['a','b','b','d'],'data1':range(4)})
right = pd.DataFrame({'key':['a','b','c'],'data2':range(3)})
result = pd.merge(left,right,on=['key'],how='right')
result.head()

,key,data1,data2
0,a,0.0,0
1,b,1.0,1
2,b,2.0,1
3,c,NaN,2


上面我们了解两种合并数据的方式，接下来我们看一下他们的应用场景：  
例如： 现在有两张表格分别存储了9月和10月份的成交信息，那么这个时候我们就可以使用concat( )将两个表沿着0轴合并。  
例如： 现在有两张表格，一个是成交信息，包含订单号、金额、客户ID等信息；第二个是客户信息，包含客户ID、姓名、电话号等信息，那么这个时候我们就可以使用merge()根据客户ID将两个表合并成一个完整的表。  
## 数据的筛选
前面的课程中，我们学习了如何获取一条数据或者连续的多条数据，但是实际工作中我们经常需要处理上万条数据，特别是合并后的数据甚至上亿条，那么我们如何能快速筛选出符合条件的数据呢？  
接下来，我们用某网站2017年用户数据为实验数据，来筛选我们想要的数据，路径为/data/mouhu_users_2017.csv。  
数据详情:id、关注的收藏夹数量、关注数量、关注者数量（粉丝数）、关注的问题数量、关注的话题数量、关注的专栏数量。  
运行下面代码，了解数据的基本情况。  

In [29]:
import pandas as pd
df = pd.read_csv('./data/mouhu_users_2017.csv', encoding="utf-8-sig")
df.head()

,_id,关注的收藏夹,关注,关注者,关注的问题,关注的话题,关注的专栏
0,587598f89f11daf90617fb7a,52,17,1,30,58,2
1,587598f89f11daf90617fb7c,27,73,15,87,26,1
2,587598f89f11daf90617fb7e,72,94,1,112,20,4
3,587598f89f11daf90617fb80,174,84,8,895,30,7
4,587598f89f11daf90617fb82,3,236,64,119,44,17


第一个需求是将关注者超过100的用户数据获取出来。  
我们先来看看小K的筛选逻辑，然后运行代码，验证筛选结果：  

In [32]:
import pandas as pd
df = pd.read_csv('./data/mouhu_users_2017.csv')
bools= df['关注者']>100
df1 = df[bools]
print(df1.shape)
df1.head(10)

(5501, 7)


,_id,关注的收藏夹,关注,关注者,关注的问题,关注的话题,关注的专栏
79,587598f89f11daf90617fc18,8,111,3049,1216,45,17
101,587598f89f11daf90617fc44,61,123,205,670,24,18
112,587598f99f11daf90617fc5a,1,804,197,830,39,26
114,587598f99f11daf90617fc5e,44,62,201,314,8,3
121,587598f99f11daf90617fc6c,44,628,6597,1566,144,69
123,587598f99f11daf90617fc70,127,74,244,2,1,35
129,587598f99f11daf90617fc7c,15,53,2381,101,23,0
155,587598f99f11daf90617fcb0,194,250,1103,10,1,19
171,587598f99f11daf90617fcd0,2,119,221,36,46,0
174,587598f99f11daf90617fcd6,12,44,1000,163,66,4


我们已经准确获取到所有关注者超过100的用户数据，下面我们看一下代码的逻辑。  
代码片段  
```
bools= df['关注者']>100
```
首先判断每个用户的关注者数量是否大于100，大于则会返回True，表示该行被标记为True，否则被标记为False。bools记录了每一行是否符合筛选条件，是一个Series对象，其中的值是bool类型。  
代码片段  
```
df1 = df[bools]
```
然后，根据bools每行的值来对df进行筛选，值为True，表示对应的行会留下，否则，则去除。  
最后打印的df1数据就是关注者超过100的用户数据。这是pandas根据某列的值进行筛选的基本逻辑  。
根据某列的值进行筛选的逻辑我们已经掌握了，如何进行多条件的联合筛选呢？  
第二个需求是：获取关注者超过300并且关注的超过100的用户数据。  
现在需要你抄写代码到下方的代码框，然后运行。  

In [34]:
import pandas as pd
df = pd.read_csv('./data/mouhu_users_2017.csv')
bool1= df['关注者']>300
bool2= df['关注']>100
df2 = df[bool1 & bool2]
df2.head()

,_id,关注的收藏夹,关注,关注者,关注的问题,关注的话题,关注的专栏
79,587598f89f11daf90617fc18,8,111,3049,1216,45,17
121,587598f99f11daf90617fc6c,44,628,6597,1566,144,69
155,587598f99f11daf90617fcb0,194,250,1103,10,1,19
228,587598f99f11daf90617fd42,48,224,1450,360,128,20
261,587598f99f11daf90617fd84,1125,842,24848,108,33,27


上面的这段代码里，我们通过了2个限制条件df['关注者']>300和 df['关注']>100，分别得到 bool1和bool2这2个Series。  
在我们的需求中，需要的数据是同时满足两个条件，所以我们使用逻辑与运算连接两个值，最后获取同时满足两个条件的值。  
### 数据的排序 
在数据获取过程中，数据的排序也是我们经常需要处理的问题。例如：我们需要找出关注者数量前十的用户信息。  
![](./picture/5-8.png)
可以使用sort_index()、sort_values()两个方法对数据进行排序，并且这两个方法Series和DataFrame都支持。  
DataFrame的sort_index( )方法是按照行索引进行排序，sort_values()可以指定具体列进行排序。  
接下来，我们使用的世界年龄抚养比率数据为行索引排序实验数据，是否还记得这个数据呢？运行下面的代码，我们一起回顾一下这个数据：  

In [39]:
import pandas as pd
df = pd.read_excel('./data/rate.xls',index_col='Country Code')
df.sort_index(inplace=True,ascending=True)
df.head()

,Country Name,1990,2000,2007,2008,2009,2010,2011,2012,2013,2014,2015,Change 1990-2015,Change 2007-2015,Unnamed: 15
Country Code,,,,,,,,,,,,,,,
ABW,Aruba,47.5008,44.4208,43.476,43.9578,44.5311,45.1572,44.9932,44.7067,44.3553,44.0703,43.9524,-3.54844,0.476409,NaN
ADO,Andorra,,,,,,,,,,,,,,NaN
AFG,Afghanistan,101.095,103.254,100,100.216,100.06,99.4598,97.6679,95.3127,92.6028,89.7738,86.9545,-14.1405,-13.0459,NaN
AGO,Angola,101.395,100.93,102.564,102.609,102.429,102.036,102.107,101.837,101.315,100.638,99.8558,-1.53897,-2.70806,NaN
ALB,Albania,61.8083,59.5859,50.863,49.6638,48.6371,,46.7203,45.8357,45.2475,44.9122,44.807,-17.0013,-6.05601,NaN


通过结果发现，原来排序这么简单！接下来我们一起分析一下代码。  
read_excel()中的参数index_col='Country Code'作用是在读取文件的时候指定Country Code这一列数据为行索引。  
inplace=True参数和我们之前见过的作用一样，用来控制是否直接对原始数据进行修改。    
ascending可以控制排序的顺序，默认值为True从小到大排列，当它被设置为False的时候就可以实现倒序排列。  
现在就需要你马上动手更改上面的代码，实现数据根据索引进行倒序排列。  
现在我们来完成第一个问题，获取某乎关注者数据前十的用户数据。那么，我们就需要根据关注者这一列数据进行排序  

In [43]:
# 数值排序
import xlrd
import pandas as pd
'''
可以使用sort_index()、sort_values()两个方法对数据进行排序，并且这两个方法Series和DataFrame都支持。
DataFrame的sort_index( )方法是按照行索引进行排序，sort_values()可以指定具体列进行排序。
接下来，我们使用的世界年龄抚养比率数据为行索引排序实验数据，是否还记得这个数据呢？运行下面的代码，我们一起回顾一下这个数据：

'''

df = pd.read_csv('./data/mouhu_users_2017.csv', encoding="utf-8-sig")
df.sort_values(by='关注者',ascending=False,inplace=True)
df.head(10)

,_id,关注的收藏夹,关注,关注者,关注的问题,关注的话题,关注的专栏
23313,5876ee159f11da03c9726155,3181,1053,585787,93,33,26
14461,5876ede89f11da03c9721c33,1041,254,356246,92,3,23
22048,5876ee0c9f11da03c9725773,41,72,351414,336,47,16
52464,5876ece59f11da03afbc71b3,1652,794,308553,23,1,112
42135,5876eb5a9f11da0399d461b7,1182,532,268353,23,1,54
63579,5878396e9f11da09c8fe21e6,10819,1412,198199,90,1,517
21740,5876ee0b9f11da03c972550b,991,479,195175,46,3,18
56721,5876ecfa9f11da03afbc92f1,164,550,164254,0,1,19
22784,5876ee129f11da03c9725d33,2011,579,148457,49,6,44
29577,587838f59f11da09bd427945,8,303,137087,477,19,15


In [44]:
import pandas as pd
'''
根据国家名称来进行排序，并且Country Code这一列被设置成了行索引。
read_excel()中的参数index_col='Country Code'作用是在读取文件的时候指定Country Code这一列数据为行索引。
inplace=True参数和我们之前见过的作用一样，用来控制是否直接对原始数据进行修改。

ascending可以控制排序的顺序，默认值为True从小到大排列，当它被设置为False的时候就可以实现倒序排列。

现在就需要你马上动手更改上面的代码，实现数据根据索引进行倒序排列。

'''
df = pd.read_excel('./data/rate.xls',index_col='Country Code')
df.sort_index(inplace=True,ascending=True)
df.head()

,Country Name,1990,2000,2007,2008,2009,2010,2011,2012,2013,2014,2015,Change 1990-2015,Change 2007-2015,Unnamed: 15
Country Code,,,,,,,,,,,,,,,
ABW,Aruba,47.5008,44.4208,43.476,43.9578,44.5311,45.1572,44.9932,44.7067,44.3553,44.0703,43.9524,-3.54844,0.476409,NaN
ADO,Andorra,,,,,,,,,,,,,,NaN
AFG,Afghanistan,101.095,103.254,100,100.216,100.06,99.4598,97.6679,95.3127,92.6028,89.7738,86.9545,-14.1405,-13.0459,NaN
AGO,Angola,101.395,100.93,102.564,102.609,102.429,102.036,102.107,101.837,101.315,100.638,99.8558,-1.53897,-2.70806,NaN
ALB,Albania,61.8083,59.5859,50.863,49.6638,48.6371,,46.7203,45.8357,45.2475,44.9122,44.807,-17.0013,-6.05601,NaN


我们成功的获取了关注者数据前十的用户数据，按照惯例我们一起分析一下代码。  
by:决定了是按数据中的哪一列进行排序，将需要按照某列排序的列名赋值给by即可。  
ascending=False:将数据按照从大到小的顺序排列。  
inplace=True:用来控制是否直接对原始数据进行修改。  
根据排序后的结果，我们发现最高的关注者数量已经达到了585787，并远远的看着第二名的356246，确有大V的潜质啊。  
### 本节总结
数据的合并、筛选和排序，是数据整理中比较重要的技能，就像将自行车变跑车，会大大提高你的工作效率，成功没有捷径，必须反复练习，勤于总结。下面我们对这节的知识点进行以下总结。  
![](./picture/5-9.png)
数据的合并、筛选和排序，是数据整理中比较重要的技能，就像将自行车变跑车，会大大提高你的工作效率，成功没有捷径，必须反复练习，勤于总结。下面我们对这节的知识点进行以下总结。


### 题目要求
现在我们有2017年度1月份和2月份的共享单车历史骑行数据，路径为/data/2017_1_data.csv和/data/2017_2_data.csv。  

### 题目讲解 
将两个csv数据合并成一个数据，并按骑行时间进行倒序排列，获取最长的骑行时间。  
参数据描述  
Duration (ms)：骑行时间， 以毫秒为单位  
Start date：开始骑行时间  
End date：结束骑行时间  
Start station：开始地点  
Start date：开始骑行时间  
End station：结束地点  
Bike number：共享单车车号  
Member type：用户类别： 会员（Member/casual非会员）  
### 书写代码

In [47]:
import pandas as pd
df1 = pd.read_csv('./data/2017_1_data.csv')
print(df1.shape)
df2 = pd.read_csv('./data/2017_2_data.csv')
print(df2.shape)
# 合并数据
df3 = pd.concat([df1,df2])
print(df3.shape)
print(df3.head())
# 倒序排列时间
df3.sort_values(by='Duration (ms)',ascending=False,inplace=True)
print(df3.head())
# 获取最长时间
long_time = df3.iloc[0]['Duration (ms)']
df3.head()
print(long_time)

(25197, 7)
(46932, 7)
(72129, 7)
  Duration (ms)      Start date        End date  \
0        221834   2017/1/1 0:00   2017/1/1 0:04   
1       1676854   2017/1/1 0:06   2017/1/1 0:34   
2       1356956   2017/1/1 0:07   2017/1/1 0:29   
3       1327901   2017/1/1 0:07   2017/1/1 0:29   
4       1636768   2017/1/1 0:07   2017/1/1 0:34   

                                  Start station                 End station  \
0                            3rd & Tingey St SE    M St & New Jersey Ave SE   
1                              Lincoln Memorial               8th & D St NW   
2   Henry Bacon Dr & Lincoln Memorial Circle NW   New York Ave & 15th St NW   
3   Henry Bacon Dr & Lincoln Memorial Circle NW   New York Ave & 15th St NW   
4                              Lincoln Memorial               8th & D St NW   

   Bike number  Member type  
0       W00869       Member  
1       W00894       Casual  
2       W21945       Casual  
3       W20012       Casual  
4       W22786       Casual  
     

### 题目要求
根据第六次全国人口普查中的常住人口数据，获取全国人口最多的10个城市。数据信息：共包含省、地区、结尾、常住人口4个字段。文件路径为/data/course_data/data_analysis/liupu.csv
### 题目讲解
难点：根据结尾字段的数据，过滤掉省，保留市。
### 编写代码

In [ ]:
import pandas as pd
df = pd.read_csv('./data/liupu.csv')
# 查看数据
print(df.head())

# series.str会将每一个数据转换成字符串
# contains()判断字符串是否含有指定子串，返回的是bool类型
bools = df['结尾'].str.contains("市")

# 根据人口数倒序排列
df = df[bools].sort_values('常住人口',ascending=False)
# 获取前十个数据
print(df.head(10))